In [1]:
import numpy as np
import matplotlib.pyplot as plt
from src.collision_operator_HG_1D import CollisionOperatorHG1D
from model.deeponet import *
import torch
from tqdm import tqdm


In [2]:
L = 10.0  # Length of the 1D domain
n_x = 1000  # Number of spatial grid points
dx = L / n_x  # Spatial grid size
sigma_t = 1.0  # Total cross-section
sigma_s = 0.5  # Scattering cross-section
source = np.zeros(n_x)  # Source term

# Angular discretization (Discrete Ordinates)
n_q = 100  # Number of discrete angles
Q = CollisionOperatorHG1D(integrator_order=n_q, anisotropy_param=0.9)
omega = Q.get_quad_pts()
w_q = Q.get_quad_weights()
# Initialize angular flux and boundary conditions
f_init = np.zeros((n_q, n_x))
f_init[:, 0] = 1.0  # Left boundary (Dirichlet boundary condition)
f_init[:, - 1] = 0.0  # Right boundary (Dirichlet boundary condition)

# Define the time step and number of time steps
dt = 0.8 * dx  # obeys cfl
t_f = 3
n_t = int(t_f/dt)
print(n_t)

375


In [3]:
gparams = {
     'name': '3_8_3_8_wo_bias',
     'seed': 0,
     'use_squeue': False,
     'dimension': 1,
     'data_file': 'legendre',
     'integration_order': 100,
     'model': 'deeponet',
     'branch_hidden': [100, 8, 8, 8],
     'trunk_hidden': [1, 8, 8, 8],
     'act': 'tanh',
     'd_out': 1,
     'use_bias': 'no',
     'use_gram': False,
     'batch_size': 0,
     'epochs': 100000,
     'lr': 0.001,
     'lambda': 0.0
}
names=[
    'wo_bias',
    'w_bias',
    'soft_lamb01',
    'hard_gram',
    'hard_special',
]
list_filename=[
    '1D/entropy/3_64_3_64_HG_wo_bias',
    '1D/entropy/3_64_3_64_HG_w_bias',
    '1D/entropy/3_64_3_64_HG_soft_lamb01',
    '1D/entropy/3_64_3_64_HG_hard_gram',
    '1D/entropy/3_64_3_64_HG_hard_special',
]

## guad pts and weights
integration_order=gparams['integration_order']
if gparams['dimension']==1:
    grid = torch.FloatTensor(Q.get_quad_pts()).reshape(-1,1).cpu()
    quad_w = torch.FloatTensor(Q.get_quad_weights()).cpu()
    size_domain=2

In [4]:
### Model
models=[]
n_basiss=[]
for idx_model, model in enumerate(list_filename):
    args=torch.load('results/'+list_filename[idx_model]+'/args.bin', map_location=torch.device('cpu'))
    gparams = args.__dict__
    branch_hidden=gparams['branch_hidden']
    trunk_hidden=gparams['trunk_hidden']
    act=gparams['act']
    output_d_out=gparams['d_out']
    if gparams['use_bias']=='no':
        use_bias=False
    else:
        use_bias=gparams['use_bias']    
    if gparams['use_bias']=='no' and gparams['use_gram']==False:
        n_basis=trunk_hidden[-1]
    else:
        n_basis=trunk_hidden[-1]+1
    use_gram=gparams['use_gram']
    ## We want a cpu model
    if gparams['model']=='deeponet':
        DeepONet=deeponet(branch_hidden, trunk_hidden, act, output_d_out, use_bias, use_gram, quad_w, size_domain).cpu()
        save_model=torch.load('results/'+model+'/final.bin', map_location=torch.device('cpu'))
        DeepONet.load_state_dict(save_model['state_dict'])
    models.append(DeepONet) 
    n_basiss.append(n_basis)

In [5]:
def get_upwind_flux(i, omega_q, f_q):
    if omega_q >= 0:
        return omega_q * f_q[i - 1]
    return omega_q * f_q[i]

In [6]:
def run_simulation(f_init, n_x, n_q, n_t, dt, dx, sigma_s, sigma_t, source, omega, model, grid, w_q, normalize):
    f_kin = np.copy(f_init)
    fluxes = np.zeros(shape=(n_q, n_x))
    # Perform time-stepping
    for k in tqdm(range(n_t - 1)):
        for i in range(1, n_x - 1):
            for q in range(n_q):
                fluxes[q, i] = get_upwind_flux(i, omega[q], f_kin[q, :])
        for i in range(1, n_x - 1):
            if normalize == False:
                _f = torch.FloatTensor(f_kin[:, i].reshape(1, 1,-1))
                collision = model(_f, grid).squeeze().detach().numpy()
            else:
                mass = (w_q @ f_kin[:, i])
                if mass==0:
                    # _f = torch.FloatTensor(f_kin[:, i].reshape(1, 1,-1))
                    # collision = model(_f, grid).squeeze().detach().numpy()
                    collision=np.zeros(f_kin[:, i].shape)
                else:
                    f_normalized = f_kin[:, i] / mass * 3.5449
                    _f = torch.FloatTensor(f_normalized.reshape(1, 1, -1))
                    collision = model(_f, grid).squeeze().detach().numpy() * mass / 3.5449
                
            for q in range(n_q):
                # Update angular flux using finite difference method
                f_kin[q, i] = f_kin[q, i] - dt / dx * (
                        fluxes[q, i + 1] - fluxes[q, i]) + dt * collision[q] + dt * source[i]

    return f_kin

In [7]:
def run_classic_simulation(f_init, n_x, n_q, n_t, dt, dx, sigma_s, sigma_t, source, omega, Q):
    f_kin = np.copy(f_init)
    fluxes = np.zeros(shape=(n_q, n_x))
    # Perform time-stepping
    for k in tqdm(range(n_t - 1)):
        for i in range(1, n_x - 1):
            for q in range(n_q):
                fluxes[q, i] = get_upwind_flux(i, omega[q], f_kin[q, :])
        for i in range(1, n_x - 1):
            collision = Q.evaluate_Q(f_kin[:, i])

            for q in range(n_q):
                # Update angular flux using finite difference method
                f_kin[q, i] = (
                    f_kin[q, i]
                    - dt / dx * (fluxes[q, i + 1] - fluxes[q, i])
                    + dt * collision[q]
                    + dt * source[i]
                )

    return f_kin

In [ ]:
f_tN = run_classic_simulation(f_init, n_x, n_q, n_t, dt, dx, sigma_s, sigma_t, source, omega, Q)

 35%|████████████████████▎                                     | 131/374 [18:29:05<9:05:04, 134.59s/it]

In [ ]:
f_res_tn = w_q @ f_tN / 2.
f_res_t0 = w_q @ f_init / 2.
import seaborn as sns

sns.set_theme()
sns.set_style("ticks")
# Plot the angular flux for one angle (e.g., the first angle)
plt.plot(np.linspace(0, L, n_x), f_res_t0)
plt.plot(np.linspace(0, L, n_x), f_res_tn)
plt.xlabel('Position (cm)')
plt.ylabel('Angular Flux')
plt.title('(Numerical) 1D-HG Discrete Ordinates with Dirichlet BC', size=15)
plt.legend(['t0', 'tN'])
plt.grid(True)
plt.tight_layout()
plt.savefig("numerical.png")

exact_f_res_t0=f_res_t0
exact_f_res_tn=f_res_tn

In [ ]:
model_idx=0
print(names[model_idx])
f_tN0 = run_simulation(f_init, n_x, n_q, n_t, dt, dx, sigma_s, sigma_t, source, omega, models[model_idx], grid, w_q, True)

sns.set_theme()
sns.set_style("ticks")

f_res_tn = w_q @ f_tN / 2.
f_res_t0 = w_q @ f_init / 2.
# Plot the angular flux for one angle (e.g., the first angle)
plt.plot(np.linspace(0, L, n_x), f_res_t0)
plt.plot(np.linspace(0, L, n_x), f_res_tn)
plt.plot(np.linspace(0, L, n_x), exact_f_res_tn, c='r', linestyle="--")
plt.xlabel('Position (cm)')
plt.ylabel('Angular Flux')
plt.title('({}) 1D-HG Discrete Ordinates with Dirichlet BC'.format(names[model_idx]), size=15)
plt.legend(['t0', 'tN','exact tN'])
plt.grid(True)
plt.xlim([0,6])
plt.tight_layout()

#plt.savefig("wo_bias.png")
print("({}) Rel error : {:.3E}".format(names[model_idx],np.mean((f_res_tn-exact_f_res_tn)**2)))

In [ ]:
model_idx=1
print(names[model_idx])
f_tN1 = run_simulation(f_init, n_x, n_q, n_t, dt, dx, sigma_s, sigma_t, source, omega, models[model_idx], grid, w_q, True)

f_res_tn = w_q @ f_tN / 2.
f_res_t0 = w_q @ f_init / 2.
# Plot the angular flux for one angle (e.g., the first angle)
plt.plot(np.linspace(0, L, n_x), f_res_t0)
plt.plot(np.linspace(0, L, n_x), f_res_tn)
plt.plot(np.linspace(0, L, n_x), exact_f_res_tn, c='r', linestyle="--")
plt.xlabel('Position (cm)')
plt.ylabel('Angular Flux')
plt.title('({}) 1D-HG Discrete Ordinates with Dirichlet BC'.format(names[model_idx]), size=15)
plt.legend(['t0', 'tN','exact tN'])
plt.grid(True)
#plt.savefig("wo_bias.png")
print("({}) Rel error : {:.3E}".format(names[model_idx],np.mean((f_res_tn-exact_f_res_tn)**2)))

In [ ]:
model_idx=2
print(names[model_idx])
f_tN2 = run_simulation(f_init, n_x, n_q, n_t, dt, dx, sigma_s, sigma_t, source, omega, models[model_idx], grid, w_q, True)

f_res_tn = w_q @ f_tN / 2.
f_res_t0 = w_q @ f_init / 2.
# Plot the angular flux for one angle (e.g., the first angle)
plt.plot(np.linspace(0, L, n_x), f_res_t0)
plt.plot(np.linspace(0, L, n_x), f_res_tn)
plt.plot(np.linspace(0, L, n_x), exact_f_res_tn, c='r', linestyle="--")
plt.xlabel('Position (cm)')
plt.ylabel('Angular Flux')
plt.title('({}) 1D-HG Discrete Ordinates with Dirichlet BC'.format(names[model_idx]), size=15)
plt.legend(['t0', 'tN','exact tN'])
plt.grid(True)
#plt.savefig("wo_bias.png")
print("({}) Rel error : {:.3E}".format(names[model_idx],np.mean((f_res_tn-exact_f_res_tn)**2)))

In [ ]:
model_idx=3
print(names[model_idx])
f_tN3 = run_simulation(f_init, n_x, n_q, n_t, dt, dx, sigma_s, sigma_t, source, omega, models[model_idx], grid, w_q, True)

f_res_tn = w_q @ f_tN / 2.
f_res_t0 = w_q @ f_init / 2.
# Plot the angular flux for one angle (e.g., the first angle)
plt.plot(np.linspace(0, L, n_x), f_res_t0)
plt.plot(np.linspace(0, L, n_x), f_res_tn)
plt.plot(np.linspace(0, L, n_x), exact_f_res_tn, c='r', linestyle="--")
plt.xlabel('Position (cm)')
plt.ylabel('Angular Flux')
plt.title('({}) 1D-HG Discrete Ordinates with Dirichlet BC'.format(names[model_idx]), size=15)
plt.legend(['t0', 'tN','exact tN'])
plt.grid(True)
#plt.savefig("wo_bias.png")
print("({}) Rel error : {:.3E}".format(names[model_idx],np.mean((f_res_tn-exact_f_res_tn)**2)))

In [ ]:

plt.plot(np.linspace(0, L, n_x), exact_f_res_tn, c='r', linestyle="--")
plt.xlabel('Position (cm)')
plt.ylabel('Angular Flux')
plt.title('({}) 1D-HG Discrete Ordinates with Dirichlet BC'.format(names[model_idx]), size=15)
plt.legend(['t0', 'tN','exact tN'])
plt.grid(True)
#plt.savefig("wo_bias.png")
print("({}) Rel error : {:.3E}".format(names[model_idx],np.mean((f_res_tn-exact_f_res_tn)**2)))

In [ ]:
model_idx=4
print(names[model_idx])
f_tN4 = run_simulation(f_init, n_x, n_q, n_t, dt, dx, sigma_s, sigma_t, source, omega, models[model_idx], grid, w_q, True)

f_res_tn = w_q @ f_tN / 2.
f_res_t0 = w_q @ f_init / 2.
# Plot the angular flux for one angle (e.g., the first angle)
plt.plot(np.linspace(0, L, n_x), f_res_t0)
plt.plot(np.linspace(0, L, n_x), f_res_tn)
plt.plot(np.linspace(0, L, n_x), exact_f_res_tn, c='r', linestyle="--")
plt.xlabel('Position (cm)')
plt.ylabel('Angular Flux')
plt.title('({}) 1D-HG Discrete Ordinates with Dirichlet BC'.format(names[model_idx]), size=15)
plt.legend(['t0', 'tN','exact tN'])
plt.grid(True)
#plt.savefig("wo_bias.png")
print("({}) Rel error : {:.3E}".format(names[model_idx],np.mean((f_res_tn-exact_f_res_tn)**2)))

In [ ]:
import seaborn as sns

f_res_tn1 = w_q @ f_tN1 / 2.
f_res_tn2 = w_q @ f_tN2 / 2.
f_res_tn3 = w_q @ f_tN3 / 2.
f_res_tn4 = w_q @ f_tN4 / 2.

sns.set_theme()
sns.set_style("ticks")

#plt.plot(np.linspace(0, L, n_x), f_res_t0)
#plt.plot(np.linspace(0, L, n_x), f_res_tn1,  marker="o")
#plt.plot(np.linspace(0, L, n_x), f_res_tn2, marker="+")
#plt.plot(np.linspace(0, L, n_x), f_res_tn3, marker="^")
plt.plot(np.linspace(0, L, n_x), f_res_tn4, marker="o")
plt.plot(np.linspace(0, L, n_x), exact_f_res_tn, c='k', linestyle="-")

ftsize = 32
plt.xlabel('x',fontsize=ftsize)
#plt.ylabel('Angular Flux',fontsize=ftsize)
#plt.title('({}) 1D-HG Discrete Ordinates with Dirichlet BC'.format(names[model_idx]), size=15)
plt.legend([ 'DeepONet', r'Baseline'],fontsize=20)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)

plt.grid(True)
plt.xlim([0,6])
plt.tight_layout()
plt.savefig("1d_simulation.pdf")
print("({}) Rel error : {:.3E}".format(names[model_idx],np.mean((f_res_tn-exact_f_res_tn)**2)))

In [ ]:
sns.set_theme()
sns.set_style("ticks")

#plt.plot(np.linspace(0, L, n_x), f_res_t0)
print(np.linalg.norm(f_res_tn1.reshape(100,1)-exact_f_res_tn.reshape(100,1), axis =1).shape)
#plt.plot(np.linspace(0, L, n_x), exact_f_res_tn, c='k', linestyle="-")
plt.plot(np.linspace(0, L, n_x), np.linalg.norm(f_res_tn1.reshape(100,1)-exact_f_res_tn.reshape(100,1), axis =1),  marker="o")
plt.plot(np.linspace(0, L, n_x), np.linalg.norm(f_res_tn2.reshape(100,1)-exact_f_res_tn.reshape(100,1), axis =1), marker="+")
plt.plot(np.linspace(0, L, n_x), np.linalg.norm(f_res_tn3.reshape(100,1)-exact_f_res_tn.reshape(100,1), axis =1), marker="^")
plt.plot(np.linspace(0, L, n_x), np.linalg.norm(f_res_tn4.reshape(100,1)-exact_f_res_tn.reshape(100,1), axis =1), marker="*")

plt.xlabel('x',fontsize=32)

plt.yscale('log')
#plt.title('({}) 1D-HG Discrete Ordinates with Dirichlet BC'.format(names[model_idx]), size=15)
plt.legend(['Vanilla','Soft constraint','Orthogonal','Bias adaption'],fontsize=20)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.grid(True)
plt.xlim([0,6])
plt.ylim([0.00001,0.01])
plt.tight_layout()
plt.savefig("1Derrors.pdf")
print("({}) Rel error : {:.3E}".format(names[model_idx],np.mean((f_res_tn-exact_f_res_tn)**2)))